In [1]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
from collections import OrderedDict
import time

import helper
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

### Loading Data

In [3]:
train_transform = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.Resize(225),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

In [4]:
train_data = datasets.ImageFolder("../Data/Cat_Dog_data/train/", transform=train_transform)
test_data = datasets.ImageFolder("../Data/Cat_Dog_data/test/", transform=test_transform)

In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

### Attempt 1

In [6]:
model = models.densenet121(pretrained=True)

In [7]:
# freeze parameters
for param in model.parameters():
    param.requires_grad = False

In [8]:
classifier = nn.Sequential(OrderedDict([
    ("fc1", nn.Linear(1024, 500)),
    ("relu", nn.ReLU()),
    ("fc2", nn.Linear(500, 2)),
    ("output", nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

In [9]:
USE_GPU = torch.cuda.is_available()
print("USE_GPU = {}".format(USE_GPU))
if USE_GPU:
    print('current_device={}'.format(torch.cuda.current_device()))

USE_GPU = True
current_device=0


In [10]:
device = "cuda"
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model.to(device)

for index, (inputs, labels) in enumerate(train_loader):
    inputs, labels = inputs.to(device), labels.to(device)
    
    start_time = time.time()
    outputs = model.forward(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if index == 3:
        break
    
end_time = time.time()

print("Device : {}\n".format(device))
print("Total Time : {}".format(end_time - start_time))
print("Time per batch : {} seconds".format((end_time - start_time) / 3))

Device : cuda

Total Time : 0.06250715255737305
Time per batch : 0.020835717519124348 seconds


### Attempt 2

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using : {}".format(device))

Using : cuda


In [12]:
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

In [13]:
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
model.to(device);

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(test_loader):.3f}.. "
                  f"Test accuracy: {accuracy/len(test_loader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 0.954.. Test loss: 0.509.. Test accuracy: 0.689
Epoch 1/1.. Train loss: 0.554.. Test loss: 0.172.. Test accuracy: 0.936
Epoch 1/1.. Train loss: 0.322.. Test loss: 0.159.. Test accuracy: 0.948
Epoch 1/1.. Train loss: 0.381.. Test loss: 0.131.. Test accuracy: 0.952
Epoch 1/1.. Train loss: 0.223.. Test loss: 0.114.. Test accuracy: 0.959
Epoch 1/1.. Train loss: 0.227.. Test loss: 0.072.. Test accuracy: 0.976
Epoch 1/1.. Train loss: 0.199.. Test loss: 0.061.. Test accuracy: 0.978
Epoch 1/1.. Train loss: 0.220.. Test loss: 0.072.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.177.. Test loss: 0.068.. Test accuracy: 0.975
Epoch 1/1.. Train loss: 0.203.. Test loss: 0.053.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.174.. Test loss: 0.053.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.168.. Test loss: 0.054.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.198.. Test loss: 0.072.. Test accuracy: 0.974
Epoch 1/1.. Train loss: 0.161.. Test loss: 0.050.. Test accuracy